# Analyzing the meter

## Load and filter the data

In [1]:
import pandas

In [2]:
#df_rv_lines = pandas.read_csv("data/rv_lines_ninth_class.csv")
df_rv_lines = pandas.read_csv("data/rv_lines_ninth_class.csv", keep_default_na=False)
#df_rv_lines.head()
#df_rv_lines.dtypes

In [3]:
# lines with meter set
df_rv_lines_filtered = df_rv_lines.query('stanza_meter != ""')

df_rv_lines_filtered

#df_rv_lines_filtered.query('stanza_meter == "Jagatī"').sort_values('line_no')

,line_no,stem,stem_type,root_guess,variant_no,weak_only,attestation_texts,language_period,root,book,...,word_gloss,text_padapatha,text_samhitapatha,meter_scansion,stanza_meter,stanza_strata,stanza_late_addition,hymn_absolute_no,hymn_addressee,hymn_group
0,01.063.02.d,iṣṇā,strong,iṣ,,False,,Earlier,iṣ 1,1,...,2.SG.PRS.IND.ACT,púraḥ iṣṇā́si= puruhūta pūrvī́ḥ,púra iṣṇā́si puruhūta pūrvī́ḥ,SS LLS SSLS LL,Triṣṭubh,A,,63,Indra,"Hymns of Nodhas, Descendant of Gotama"
1,01.063.04.b,ubhnā,strong,ubh,,False,V.,Earlier,ubh,1,...,3.SG.PRS.INJ.ACT,vr̥trám yát vajrin= vr̥ṣakarman ubhnā́ḥ,vr̥tráṁ yád vajrin vr̥ṣakarman ubhnā́ḥ,LL L LL SSLS LL,Triṣṭubh,A,,63,Indra,"Hymns of Nodhas, Descendant of Gotama"
3,04.018.12.d,kṣiṇā,strong,kṣi,,False,V.B.,Earlier,kṣī,4,...,2.SG.IPRF.IND.ACT,yát prá ákṣiṇāḥ= pitáram pādagŕ̥hya,yát prā́kṣiṇāḥ pitáram pādagŕ̥hya,L LSL SSL LSLS,Triṣṭubh,P,"['Grassmann (G)', 'Arnold (C1)']",314,"Dialogue Between Indra, Aditi and Vamadeva",Hymns to Indra
4,10.027.04.d,kṣiṇā,strong,kṣi,,False,V.B.,Earlier,kṣī,10,...,1.SG.PRS.INJ.ACT,prá tám kṣiṇām= párvate-_ pādagŕ̥hya,prá táṁ kṣiṇām párvate pādagŕ̥hya,S L SL LSL LSLS,Triṣṭubh,P,['Arnold (C1)'],853,Indra,The Vasukra Hymns
5,10.027.13.c,kṣiṇā,strong,kṣi,,False,V.B.,Earlier,kṣī,10,...,3.SG.PRS.IND.ACT,ā́sīnaḥ ūrdhvā́m= upási} kṣiṇāti,ā́sīna ūrdhvā́m upási kṣiṇāti,LLS LL SSL SLS,Triṣṭubh,P,['Arnold (C1)'],853,Indra,The Vasukra Hymns
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,08.063.07.c,str̥ṇā,strong,str̥,,False,,Earlier & Later,str̥̄,8,...,3.SG.IPRF.IND.ACT,ástr̥ṇāt barháṇā vipáḥ,ástr̥ṇād barháṇā vipó,LSL LSL SL,Aṇuṣṭubh,A,,672,Indra,Hymns of Pragatha and Other Kanva-Sons
343,07.073.02.c,aśnī,weak,aśⁱ,1,True,E.,Later,aśⁱ,7,...,2.DU.PRS.OPT.ACT,aśnītám mádhvaḥ= aśvinau} upāké-_,aśnītám mádhvo aśvinā upāká,LLL LL LSL SLS,Triṣṭubh,S,,589,Asvins,Hymns to the Asvins
344,09.067.31.c,aśnā,strong,aśⁱ,1,True,E.,Later,aśⁱ,9,...,3.SG.PRS.IND.ACT,sárvam sá pūtám aśnāti,*sárvaṁ sá pūtám aśnāti,LL S LS LLS,Aṇuṣṭubh,P,"['Grassmann (G)', 'Oldenberg (O)', 'Arnold (C2...",779,Soma and Other Gods,Tirasci and Other Poets
345,10.085.03.d,aśnā,strong,aśⁱ,1,True,E.,Later,aśⁱ,10,...,3.SG.PRS.IND.ACT,ná tásya aśnāti káḥ caná,ná tásyāśnāti káś caná,S LLLS L SS,Aṇuṣṭubh,P,"['Grassmann (G)', 'Arnold (C2)']",911,Wedding Hymn,Hymns of Manyu


## Syllabification and scansion

TODO check if avagraha (o_' / o_a) is present in our lines of interest and decide how to deal with the 'o' vowel 

TODO also mark when vnh text has [bolle](https://lrc.la.utexas.edu/books/rigveda/RV00#bolle) (@), as in: https://vedaweb.uni-koeln.de/rigveda/view/id/9.11.6 

In [19]:
# from meter.py, in the same folder as this notebook
import meter

# useful during testing to pick up changes in the file
import importlib
importlib.reload(meter)

def add_meter_analysis(line):        
    analysis = meter.analyze(line["text_samhitapatha"], line["stanza_meter"])
    #print(analysis)
    
    # TODO rename this as the default meter scansion
    line["meter_scansion_custom"] = analysis["scansion"]
    line["meter_syllables_count"] = analysis["no_of_syllables"]
    line["meter_syllables"] = analysis["syllables"]
    line["meter_scansions_match"] = analysis["scansion_syllables"] == meter.clean_meter_scansion(line["meter_scansion"])
    
    if not line["meter_scansions_match"]:
        print(
            f"{line['line_no']}:", 
            f"{line['meter_scansion_custom']} ≠ {line['meter_scansion']}",
            f"[{line['text_samhitapatha']}]"
        )
    
    return line

df_rv_lines_filtered_annotated = df_rv_lines_filtered \
    .apply(add_meter_analysis, axis=1)

df_rv_lines_filtered_annotated.head()

03.049.02.d: SLSL ,SSL|_LS LL ≠ SLSL SSL LL LL [pr̥thujráyā aminād ā́yu@ dásyoḥ]
09.011.06.b: LL_S_L|SLSS ≠ LL SL SLSL [dadhnéd abhí śriṇītana@]


,line_no,stem,stem_type,root_guess,variant_no,weak_only,attestation_texts,language_period,root,book,...,stanza_meter,stanza_strata,stanza_late_addition,hymn_absolute_no,hymn_addressee,hymn_group,meter_scansion_custom,meter_syllables_count,meter_syllables,meter_scansions_match
0,01.063.02.d,iṣṇā,strong,iṣ,,False,,Earlier,iṣ 1,1,...,Triṣṭubh,A,,63,Indra,"Hymns of Nodhas, Descendant of Gotama","SS LLS ,SS|LS LL",11,"[pú, ra, iṣ, ṇā́, si, pu, ru, hū, ta, pūr, vī́ḥ]",True
1,01.063.04.b,ubhnā,strong,ubh,,False,V.,Earlier,ubh,1,...,Triṣṭubh,A,,63,Indra,"Hymns of Nodhas, Descendant of Gotama","LL L LL ,SS|LS_LL",11,"[vr̥t, ráṁ, yád, vaj, rin, vr̥, ṣa, kar, ma, n...",True
3,04.018.12.d,kṣiṇā,strong,kṣi,,False,V.B.,Earlier,kṣī,4,...,Triṣṭubh,P,"['Grassmann (G)', 'Arnold (C1)']",314,"Dialogue Between Indra, Aditi and Vamadeva",Hymns to Indra,"L LSL ,SSL |LSLS",11,"[yát, prā́k, ṣi, ṇāḥ, pi, tá, ram, pā, da, gŕ̥...",True
4,10.027.04.d,kṣiṇā,strong,kṣi,,False,V.B.,Earlier,kṣī,10,...,Triṣṭubh,P,['Arnold (C1)'],853,Indra,The Vasukra Hymns,"S L SL ,LSL |LSLS",11,"[prá, táṁ, kṣi, ṇām, pár, va, te, pā, da, gŕ̥h...",True
5,10.027.13.c,kṣiṇā,strong,kṣi,,False,V.B.,Earlier,kṣī,10,...,Triṣṭubh,P,['Arnold (C1)'],853,Indra,The Vasukra Hymns,LLS LL_SS|_LSLS,11,"[ā́, sī, na, ūr, dhvā́, m u, pá, si k, ṣi, ṇā,...",True


### Checking our scansion yields same final result as vedaweb

In [23]:
df_rv_lines_filtered_annotated[["line_no", "text_padapatha", "text_samhitapatha", "meter_scansion", "meter_scansion_custom", "meter_scansions_match"]] \
  .query('meter_scansions_match == False')

,line_no,text_padapatha,text_samhitapatha,meter_scansion,meter_scansion_custom,meter_scansions_match
58,03.049.02.d,pr̥thujráyāḥ= amināt ā́yuḥ dásyoḥ,pr̥thujráyā aminād ā́yu@ dásyoḥ,SLSL SSL LL LL,"SLSL ,SSL|_LS LL",False
111,09.011.06.b,dadhnā́ ít abhí śrīṇītana,dadhnéd abhí śriṇītana@,LL SL SLSL,LL_S_L|SLSS,False
